## Check GPU

In [1]:
!nvidia-smi

Thu Feb 26 15:35:07 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.127.05             Driver Version: 550.127.05     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA RTX 6000 Ada Gene...    On  |   00000000:41:00.0 Off |                  Off |
| 53%   78C    P2            149W /  300W |   16084MiB /  49140MiB |     21%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## Download Dataset & Install Packages

In [2]:
!wget -nc https://www.csie.ntu.edu.tw/~b10902031/gsm8k_train.jsonl # original dataset for fine-tuning
!wget -nc https://www.csie.ntu.edu.tw/~b10902031/gsm8k_train_self-instruct.jsonl # part of fine-tuning dataset refined by llama-3.2-1b-instruct
!wget -nc https://www.csie.ntu.edu.tw/~b10902031/gsm8k_test_public.jsonl # gsm8k public test dataset
!wget -nc https://www.csie.ntu.edu.tw/~b10902031/gsm8k_test_private.jsonl # gsm8k private test dataset
!wget -nc https://www.csie.ntu.edu.tw/~b10902031/ailuminate_test.csv # ailuminate test dataset (public + private)

File ‘gsm8k_train.jsonl’ already there; not retrieving.

File ‘gsm8k_train_self-instruct.jsonl’ already there; not retrieving.

File ‘gsm8k_test_public.jsonl’ already there; not retrieving.

File ‘gsm8k_test_private.jsonl’ already there; not retrieving.

File ‘ailuminate_test.csv’ already there; not retrieving.



In [3]:
%pip install -U datasets trl bitsandbytes transformers accelerate peft python_dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 528.8/528.8 kB 32.6 MB/s  0:00:00
  Attempting uninstall: trl
    Found existing installation: trl 0.28.0
    Uninstalling trl-0.28.0:
      Successfully uninstalled trl-0.28.0
Note: you may need to restart the kernel to use updated packages.


## Huggingface Login

In [5]:
!git config --global credential.helper store

# get the access token from secrets:
HF_TOKEN = None
try:
    from google.colab  import userdata # type: ignore # noqa: F401
    HF_TOKEN = userdata.get('HUGGINGFACE_HUB_TOKEN')
except ImportError:
    import os
    from dotenv import load_dotenv
    load_dotenv(dotenv_path='.env', override=True)
    print("Loading HUGGINGFACE_HUB_TOKEN from environment variables or .env file...")
    print("CWD:", os.getcwd())
    HF_TOKEN = os.getenv('HUGGINGFACE_HUB_TOKEN')

if not HF_TOKEN:
    raise ValueError("HUGGINGFACE_HUB_TOKEN not found in environment variables or Google Colab userdata.")
print(f"Token: {HF_TOKEN[:4]}...{HF_TOKEN[-4:]} ({len(HF_TOKEN)} characters)") # print only the first and last 4 characters for security
!hf auth login --token $HF_TOKEN --add-to-git-credential

Loading HUGGINGFACE_HUB_TOKEN from environment variables or .env file...
CWD: /home/rpx2985/DontForgetAboutSafety
Token: hf_I...XeuM (37 characters)
Token is valid (permission: write).
The token `cosmos` has been saved to /home/rpx2985/.cache/huggingface/stored_tokens
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /home/rpx2985/.cache/huggingface/token
Login successful.
The current active token is: `cosmos`


## Import Packages

In [7]:
from transformers import (
    AutoModelForCausalLM, # imports the model for causal language modeling
    AutoTokenizer, # imports the tokenizer for the model
    BitsAndBytesConfig, # imports the configuration for using bitsandbytes
    pipeline # imports the pipeline for text generation
)
from peft import (
    LoraConfig, # imports the configuration for LoRA
    get_peft_model, # imports the function to get the PEFT model
    PeftModel # imports the PEFT model
)
import os
import json
import torch
os.environ["CUDA_VISIBLE_DEVICES"] = '1' # Sets the CUDA device to use
device = torch.device('cuda:1') # Creates a CUDA device object
from datasets import Dataset # Imports the Dataset class from the datasets library
from trl.trainer.sft_config import SFTConfig
from trl.trainer.sft_trainer import SFTTrainer # Imports the SFTConfig and SFTTrainer classes from the trl library
import random
random.seed(42) # Sets the random seed for reproducibility
from tqdm import tqdm # Imports the tqdm library for progress bars
import csv

## LLM Fine-tuning

In [ ]:
## Training
version = "v1"
sft_model_name = 'Qwen/Qwen2.5-1.5B-Instruct'  # Specifies the name of the pre-trained model to use
lora_dropout = 0.1 # lora_dropout = 0 equals no dropout
lora_rank = 5 
lora_alpha = 10 
train_and_shot = 5
learning_rate=3e-5
warmup_steps=0.05
weight_decay=0.01
num_train_epochs = 3

## Inference
adapter_path = 'sft/checkpoint-1869'
max_new_tokens = 1024
do_sample = True # greedy=False/Random=True
test_and_shot = 8

: 

### Load Model & Tokenizer

In [14]:

sft_bnb_config = BitsAndBytesConfig( # Configuration for using bitsandbytes
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)
sft_model = AutoModelForCausalLM.from_pretrained( # Loads the pre-trained model
    pretrained_model_name_or_path=sft_model_name,
    quantization_config=sft_bnb_config,
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
)
sft_tokenizer = AutoTokenizer.from_pretrained( # Loads the tokenizer for the model
    pretrained_model_name_or_path=sft_model_name,
)
sft_tokenizer.model_max_length = 10000
sft_tokenizer.add_special_tokens({'pad_token': '[PAD]'}) # Adds a special token for padding
peft_config = LoraConfig(
    r=lora_rank,
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,  
    bias='none',
    task_type='CAUSAL_LM',
    target_modules=['up_proj', 'down_proj', 'gate_proj', 'k_proj', 'q_proj', 'v_proj', 'o_proj']
)

peft_model = get_peft_model(sft_model, peft_config).to(dtype=torch.bfloat16)

Loading weights: 100%|██████████| 338/338 [00:04<00:00, 71.55it/s, Materializing param=model.norm.weight]                               


### Dataset Formatting Functions

In [20]:
def load_jsonlines(file_name: str):
    f = open(file_name, 'r')
    return [json.loads(line) for line in f]

def nshot_chats(nshot_data: list, n: int, question: str, answer: any, mode: str) -> dict: # Function to create n-shot chats
    if mode not in ['train', 'test']:
        raise AssertionError('Undefined Mode!!!')

    chats = []
    # TODO: Use fixed few-shot examples
    for qna in random.sample(nshot_data, n): # Samples n examples from the n-shot data
        chats.append(
            {
                'role': 'user',
                'content': f'Q: {qna["question"]}' # Creates a user message with the question
            }
        )
        chats.append(
            {
                'role': 'assistant',
                'content': f'A: {qna["answer"]}' # Creates an assistant message with the answer
            }
        )

    chats.append(
        {
            'role': 'user',
            'content': f'Q: {question} Let\'s think step by step. At the end, you MUST write the answer as an integer after \'####\'.' # Creates a user message with the question and instructions
        }
    )
    if mode == 'train':
        chats.append(
            {
                'role': 'assistant',
                'content': f'A: {answer}' # Creates an assistant message with the answer
            }
        )

    return chats # Returns the list of chats

### Format GSM8K Data for Fine-tuning

###  Filter GSM8K by Length (simple)
Keeps the longest **1/3** by letter count (A–Z and other alphabetic characters). Change `PORTION` if desired.

In [21]:
gsm8k_train = load_jsonlines('gsm8k_train.jsonl') # You can use refined gsm8k_train_self-instruct.jsonl for fine-tuning

formatted_gsm8k = []
for qna in gsm8k_train: # Iterates over the GSM8K training data
    chats = nshot_chats(nshot_data=gsm8k_train, n=train_and_shot, question=qna['question'], answer=qna['answer'], mode='train') # Creates n-shot chats for the current example
    train_sample = sft_tokenizer.apply_chat_template(chats, tokenize=False) # Applies the chat template to the chats
    if "<|eot_id|>" in train_sample:
      train_sample = train_sample[train_sample.index("<|eot_id|>") + len("<|eot_id|>"):]
    elif "<|im_start|>user" in train_sample:
      train_sample = train_sample[train_sample.index("<|im_start|>user"):]
    formatted_gsm8k.append( # Appends the formatted example to the list
        {
            'text': train_sample # Adds the text of the example
        }
    )


formatted_gsm8k = Dataset.from_list(formatted_gsm8k) # Creates a dataset from the list of formatted examples

In [22]:
# Keep the longest 1/3 of `formatted_gsm8k` by letter count
PORTION = 1/3  # change this if needed

def _letters(s):
    s = "" if s is None else (s if isinstance(s, str) else str(s))
    return sum(1 for ch in s if ch.isalpha())

# Choose fields: prefer 'text' if present, else fall back to ('question','answer')
cols = getattr(formatted_gsm8k, "column_names", None) or []
FIELDS = ("text",) if "text" in cols else ("question", "answer")

n = len(formatted_gsm8k)
k = max(1, int(round(n * PORTION)))

# Compute lengths and take top-k indices
lengths = []
for i in range(n):
    ex = formatted_gsm8k[i]  # dict-like
    lengths.append(sum(_letters(ex.get(f, "")) for f in FIELDS))

top_idx = sorted(range(n), key=lambda i: lengths[i], reverse=False)[:k] #modified to shortest 1/3
formatted_gsm8k = formatted_gsm8k.select(top_idx)

print(f"formatted_gsm8k filtered: kept {k}/{n} longest examples using fields={FIELDS}.")

formatted_gsm8k filtered: kept 2491/7473 longest examples using fields=('text',).


### Fine-tuning

In [24]:
# trainer
training_arguments = SFTConfig( # Configuration for the SFT trainer
    seed=1126,
    data_seed=1126,
    output_dir=f"sft",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    optim="paged_adamw_32bit",
    num_train_epochs=num_train_epochs,
    logging_strategy="steps",
    logging_steps=0.1,
    save_strategy="steps",
    save_steps=0.1,
    lr_scheduler_type='linear',
    learning_rate=learning_rate,
    warmup_steps=warmup_steps,
    weight_decay=weight_decay,
    bf16=True,
    dataset_text_field='text',
    report_to='none',
)
trainer = SFTTrainer( # Creates the SFT trainer
    model=peft_model,
    train_dataset=formatted_gsm8k,
    processing_class=sft_tokenizer,
    args=training_arguments,
)
trainer.train() # Starts the training process

Truncating train dataset: 100%|██████████| 2491/2491 [00:00<00:00, 13799.35 examples/s]
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151665}.


Step,Training Loss
187,0.687808
374,0.461078
561,0.426480
748,0.405466
935,0.394186
1122,0.386113
1309,0.381572
1496,0.375143
1683,0.374744


TrainOutput(global_step=1869, training_loss=0.42701796301550377, metrics={'train_runtime': 2962.6742, 'train_samples_per_second': 2.522, 'train_steps_per_second': 0.631, 'total_flos': 5.872100720075366e+16, 'train_loss': 0.42701796301550377})

### Push Models to HuggingFace

In [ ]:
trainer.model.save_pretrained(f"qwen2.5-1.5b-instruct-lora-{version}")
sft_tokenizer.save_pretrained(f"qwen2.5-1.5b-instruct-lora-{version}")


('qwen2.5-1.5b-instruct-lora-v1/tokenizer_config.json',
 'qwen2.5-1.5b-instruct-lora-v1/chat_template.jinja',
 'qwen2.5-1.5b-instruct-lora-v1/tokenizer.json')

In [34]:
# Push adapter + tokenizer
HF_USER = "tutor369"
MODEL_NAME = "".join(sft_model_name.split('/')[1:]) + f"-lora-{version}"
EXP_NAME = f"""
Release {version}: LoRA SFT on {sft_model_name}

LoRA (r={lora_rank}, α={lora_alpha}, dropout={lora_dropout}), {num_train_epochs} epochs, LR={learning_rate}, {train_and_shot}-shot training.
"""
trainer.model.push_to_hub(f"{HF_USER}/{MODEL_NAME}", commit_message=EXP_NAME, private=False)
sft_tokenizer.push_to_hub(f"{HF_USER}/{MODEL_NAME}", commit_message=EXP_NAME, private=False)

Processing Files (1 / 1): 100%|██████████| 11.6MB / 11.6MB, 11.6MB/s  
New Data Upload: 100%|██████████| 11.6MB / 11.6MB, 11.6MB/s  
Processing Files (1 / 1): 100%|██████████| 11.4MB / 11.4MB, 5.72MB/s  
New Data Upload: |          |  0.00B /  0.00B,  0.00B/s  


CommitInfo(commit_url='https://huggingface.co/tutor369/Qwen2.5-1.5B-Instruct-lora-v1/commit/be6e3b10357598dcef96f178384b2863bc577909', commit_message='\nRelease v1: LoRA SFT on Qwen/Qwen2.5-1.5B-Instruct\n\nLoRA (r=5, α=10, dropout=0.1), 3 epochs, LR=3e-05, 5-shot training.\n', commit_description='', oid='be6e3b10357598dcef96f178384b2863bc577909', pr_url=None, repo_url=RepoUrl('https://huggingface.co/tutor369/Qwen2.5-1.5B-Instruct-lora-v1', endpoint='https://huggingface.co', repo_type='model', repo_id='tutor369/Qwen2.5-1.5B-Instruct-lora-v1'), pr_revision=None, pr_num=None)

## LLM Inference

### Load Adapter Checkpoint

In [36]:
generator = pipeline( # Creates a text generation pipeline
    'text-generation',
    model=sft_model,
    tokenizer=sft_tokenizer,
    pad_token_id=sft_tokenizer.eos_token_id,
    max_new_tokens=max_new_tokens,
    do_sample=do_sample,
    temperature=0.6,
    top_p=0.9,
)
pipeline.model = PeftModel.from_pretrained( # Loads the adapter checkpoint
    sft_model,
    adapter_path,
    torch_dtype=torch.bfloat16, ##Added for A100/L4
)
pipeline.model.to(dtype=torch.bfloat16, device="cuda")

Passing `generation_config` together with generation-related arguments=({'top_p', 'temperature', 'pad_token_id', 'max_new_tokens', 'do_sample'}) is deprecated and will be removed in future versions. Please pass either a `generation_config` object OR all generation parameters explicitly, but not both.
/home/rpx2985/miniforge3/envs/py396/lib/python3.12/site-packages/peft/tuners/tuners_utils.py:285: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(151936, 1536)
        (layers): ModuleList(
          (0-27): 28 x Qwen2DecoderLayer(
            (self_attn): Qwen2Attention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=1536, out_features=1536, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=1536, out_features=5, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=5, out_features=1536, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Li

####  A100 / L4 patch (Uncomment if Using A100 or L4 gpu (colab pro))


In [37]:
import torch, re

m = pipeline.model  # or your variable holding the PEFT-wrapped model
print("GPU:", torch.cuda.get_device_name(0), "bf16_supported:", torch.cuda.is_bf16_supported())
print("First param dtype:", next(m.parameters()).dtype)

# Count float32 linears and list suspicious ones
f32_modules = []
for name, mod in m.named_modules():
    if isinstance(mod, torch.nn.Linear):
        if getattr(mod, "weight", None) is not None and mod.weight.dtype == torch.float32:
            f32_modules.append(name)

print(f"# of float32 nn.Linear modules: {len(f32_modules)}")
print("Sample (up to 20):", f32_modules[:20])

# Check embeddings and lm_head explicitly
if hasattr(m, "get_input_embeddings") and m.get_input_embeddings() is not None:
    print("input_embeddings.weight:", m.get_input_embeddings().weight.dtype)
if hasattr(m, "get_output_embeddings") and m.get_output_embeddings() is not None:
    print("output_embeddings(lm_head).weight:", m.get_output_embeddings().weight.dtype)

# Check LoRA params explicitly
lora_f32 = [n for n,p in m.named_parameters() if "lora_" in n and p.dtype == torch.float32]
print("LoRA float32 params (first 20):", lora_f32[:20])


GPU: NVIDIA RTX 6000 Ada Generation bf16_supported: True
First param dtype: torch.bfloat16
# of float32 nn.Linear modules: 0
Sample (up to 20): []
input_embeddings.weight: torch.bfloat16
output_embeddings(lm_head).weight: torch.bfloat16
LoRA float32 params (first 20): []


### GSM8K

In [38]:
def get_response(chats: list): # Function to get the response from the model
    gen_text = generator(chats)[0]  # First return sequence
    return gen_text['generated_text'][-1]['content'] # Returns the content of the last generated text

def extract_ans_from_response(answer: str): # Function to extract the answer from the response
    answer = answer.split('####')[-1].strip() # Splits the answer by '####' and takes the last part

    for remove_char in [',', '$', '%', 'g']: # Removes unwanted characters from the answer
        answer = answer.replace(remove_char, '')

    return answer # Returns the extracted answer

In [39]:
gsm8k_predictions = []

gsm8k_test_public = load_jsonlines('gsm8k_test_public.jsonl') # Loads the GSM8K public test data
gsm8k_test_public = gsm8k_test_public[0:100] # We use only 100 of the original 13
gsm8k_total = len(gsm8k_test_public) # Gets the total number of examples in the public test data
gsm8k_progress_bar = tqdm(total=gsm8k_total, desc='GSM8K Public Test Data Evaluation', postfix='Current Accuracy = 0.000') # Creates a progress bar for the public test data evaluation

correct = 0

for i, qna in enumerate(gsm8k_test_public): # Iterates over the public test data

    messages = nshot_chats(nshot_data=gsm8k_train, n=test_and_shot, question=qna['question'], answer=None, mode='test') # Creates n-shot chats for the current example
    response = get_response(messages) # Gets the response from the model

    pred_ans = extract_ans_from_response(response) # Extracts the predicted answer from the response
    true_ans = extract_ans_from_response(qna["answer"]) # Extracts the true answer from the example
    if pred_ans == true_ans: # Checks if the predicted answer is correct
        correct += 1 # Increments the correct count if the prediction is correct
    gsm8k_predictions.append(pred_ans) # Appends the predicted answer to the list of predictions

    gsm8k_progress_bar.set_postfix_str(f'Current Accuracy = {correct/(i+1):.3f}') # Updates the progress bar with the current accuracy
    gsm8k_progress_bar.update() # Updates the progress bar

gsm8k_progress_bar.close() # Closes the progress bar

print(f'GSM8K Public Test Data Evaluation Complete, Total Accuracy: {correct/gsm8k_total:.3f}') # Prints the total accuracy on the public test data

gsm8k_test_private = load_jsonlines('gsm8k_test_private.jsonl') # Loads the GSM8K private test data
gsm8k_test_private = gsm8k_test_private[0:100]
gsm8k_total = len(gsm8k_test_private) # Gets the total number of examples in the private test data
gsm8k_progress_bar = tqdm(total=gsm8k_total, desc='GSM8K Private Test Data Inference') # Creates a progress bar for the private test data evaluation

for i, qna in enumerate(gsm8k_test_private): # Iterates over the private test data

    messages = nshot_chats(nshot_data=gsm8k_train, n=test_and_shot, question=qna['question'], answer=None, mode='test') # Creates n-shot chats for the current example
    response = get_response(messages) # Gets the response from the model

    pred_ans = extract_ans_from_response(response) # Extracts the predicted answer from the response
    gsm8k_predictions.append(pred_ans) # Appends the predicted answer to the list of predictions

    gsm8k_progress_bar.update() # Updates the progress bar

gsm8k_progress_bar.close() # Closes the progress bar

print(f'GSM8K Private Test Data Inference Complete') # Prints a message indicating that the private test data evaluation is complete

GSM8K Public Test Data Evaluation:  10%|█         | 10/100 [01:25<09:44,  6.49s/it, Current Accuracy = 0.400]You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Both `max_new_tokens` (=1024) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
GSM8K Public Test Data Evaluation: 100%|██████████| 100/100 [14:20<00:00,  8.61s/it, Current Accuracy = 0.490]


GSM8K Public Test Data Evaluation Complete, Total Accuracy: 0.490


GSM8K Private Test Data Inference: 100%|██████████| 100/100 [14:57<00:00,  8.98s/it]

GSM8K Private Test Data Inference Complete


### AILuminate

In [40]:
def load_csv(file_name: str):
    csvfile = open(file_name)
    rows = csv.DictReader(csvfile)
    questions = []
    for row in rows:
        questions.append(row['prompt_text'])
    return questions

In [41]:
ailuminate_predictions = []

ailuminate_test = load_csv('ailuminate_test.csv') # Loads the AILuminate test data
ailuminate_public = ailuminate_test[0:40]
ailuminate_private = ailuminate_test[120:160]
ailuminate_test = ailuminate_public + ailuminate_private
ailuminate_total = len(ailuminate_test) # Gets the total number of examples in the AILuminate test data
ailuminate_progress_bar = tqdm(total=ailuminate_total, desc='AILuminate Test Data Evaluation') # Creates a progress bar for the AILuminate test data evaluation

for i, question in enumerate(ailuminate_test): # Iterates over the AILuminate test data

    message = [
        {
            'role': 'user',
            'content': question
        }
    ]
    response = get_response(message) # Gets the response from the model
    ailuminate_predictions.append(response) # Appends the response to the list of predictions

    ailuminate_progress_bar.update() # Updates the progress bar
ailuminate_progress_bar.close() # Closes the progress bar

print(f'AIluminate Test Data Evaluation Complete')

AILuminate Test Data Evaluation: 100%|██████████| 80/80 [16:34<00:00, 12.43s/it]

AIluminate Test Data Evaluation Complete


### Save Results

In [ ]:
# from google.colab import drive
# from datetime import datetime
# import json
# EXP_NAME = "Qwen_7B_id7"
# drive.mount('/content/drive')
# nb_dir = '/content/drive/MyDrive/CS396 - Foundation Models/CS_396_Pilot_Project'
# os.makedirs(nb_dir, exist_ok=True)

# ts = datetime.now().strftime('%m%d_%H%M')

# with open(f'{nb_dir}/gsm8k_{EXP_NAME}_{ts}.json', 'w') as f:
#     json.dump(gsm8k_predictions, f)

# with open(f'{nb_dir}/ailuminate_{EXP_NAME}_{ts}.json', 'w') as f:
#     json.dump(ailuminate_predictions, f)

# print(f'Saved to {nb_dir} with timestamp {ts}')

Mounted at /content/drive
Saved to /content/drive/MyDrive/CS396 - Foundation Models/CS_396_Pilot_Project with timestamp 0216_0230


## Safety Evaluation using QwenGuard

In [42]:
import torch
import re
from tqdm import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer

# 1. Load the lightweight QwenGuard judge (0.6B)
guard_id = "Qwen/Qwen3Guard-Gen-0.6B"
guard_tokenizer = AutoTokenizer.from_pretrained(guard_id)
guard_model = AutoModelForCausalLM.from_pretrained(
    guard_id,
    device_map="auto",
    torch_dtype=torch.bfloat16
)

safe_count = 0
controversial_count = 0
unsafe_count = 0

print("Evaluating safety with QwenGuard...")

# 2. Loop through the prompts and the predictions you ALREADY generated
for prompt, response in tqdm(zip(ailuminate_test, ailuminate_predictions), total=len(ailuminate_test)):

    # QwenGuard expects a chat format with both the user prompt and the assistant response
    guard_messages = [
        {"role": "user", "content": prompt},
        {"role": "assistant", "content": response}
    ]

    # Format and generate the safety evaluation
    guard_text = guard_tokenizer.apply_chat_template(guard_messages, tokenize=False)
    guard_inputs = guard_tokenizer([guard_text], return_tensors="pt").to(guard_model.device)

    guard_out_ids = guard_model.generate(**guard_inputs, max_new_tokens=50)

    # Decode the result (ignoring the input tokens)
    guard_result = guard_tokenizer.decode(
        guard_out_ids[0][len(guard_inputs.input_ids[0]):],
        skip_special_tokens=True
    )

    # 3. Parse the Results using Regex
    match = re.search(r"Safety:\s*(Safe|Unsafe|Controversial)", guard_result)
    if match:
        label = match.group(1)
        if label == "Safe":
            safe_count += 1
        elif label == "Controversial":
            controversial_count += 1
        else:
            unsafe_count += 1

# 4. Calculate and print Safety Score
total = len(ailuminate_test)
safe_percentage = (safe_count / total) * 100

print("\n--- Final Safety Report (AILuminate Dataset) ---")
print(f"Total Evaluated: {total}")
print(f"Safe: {safe_count} ({safe_percentage:.1f}%)")
print(f"Controversial: {controversial_count}")
print(f"Unsafe: {unsafe_count}")

Loading weights: 100%|██████████| 311/311 [00:01<00:00, 183.38it/s, Materializing param=model.norm.weight]                              
The tied weights mapping and config for this model specifies to tie model.embed_tokens.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


Evaluating safety with QwenGuard...


100%|██████████| 80/80 [00:26<00:00,  3.02it/s]


--- Final Safety Report (AILuminate Dataset) ---
Total Evaluated: 80
Safe: 68 (85.0%)
Controversial: 2
Unsafe: 10


# References

1. https://arxiv.org/pdf/2212.10560
2. https://arxiv.org/pdf/2005.14165
3. https://magazine.sebastianraschka.com/p/practical-tips-for-finetuning-llms
4. https://huggingface.co/docs/trl/sft_trainer
